# RU29

This is the start of a notebook to plot the RU29 data ourselves.

It uses a python packages called `erddapy` to connect to the glider data server. Before you run this for the first time you need to install `erddapy` in your python environment with the following commands in your terminal/powershell:

`conda install --channel conda-forge erddapy`

In [ ]:
import xarray as xr
from erddapy import ERDDAP
import matplotlib.pyplot as plt

import cartopy.crs as ccrs

import numpy as np



# Connect to glider data server
This code below uses `erddapy` to create a link to the glider data, then uses xarray to download it. This can take a little while! It also doesn't keep the dat on your computer, just loads it into memory for plotting. It will give you the most up to date data whenever you run it

In [ ]:

# this creates a link to the RUCOOL server:
e = ERDDAP(
    server="http://slocum-data.marine.rutgers.edu/erddap",
    protocol="tabledap",
    response="nc",
)
e2 = ERDDAP(
    server="http://slocum-data.marine.rutgers.edu/erddap",
    protocol="tabledap",
    response="nc",
)
# this points at the RU29 glider deployment
# e.dataset_id = 'ru29-20200908T1623-profile-raw-rt'

# get the science data:
e.dataset_id = 'ru29-20200908T1623-profile-sci-rt'

#get the engineering data:
e2.dataset_id = "ru29-20200908T1623-trajectory-raw-rt"

# this connects to the data and load into an xarray
ds = e.to_xarray()
ds2 = e2.to_xarray()
# ds = e.to_pandas()


In [ ]:
# print out the long list of data, dims, coords, variables

ds

# Plot glider dives

In [ ]:
# make the figure biggger:
fig = plt.figure(figsize=(10,5))

plt.plot(ds.time, ds.depth, marker='.', linestyle='')
plt.ylim([1000, 0])

# you can zoom in on some time if you want:
plt.xlim(['2020-10-01', '2020-10-05'])

# this thing makes the dates axis look better:
fig.autofmt_xdate()

# Science plots

In [ ]:

plt.figure()
plt.plot(ds.potential_temperature, ds.depth, '.')
plt.ylabel('depth')
plt.xlabel('potential temperature')
plt.ylim([1000, 0])


# warning, scatter can take a long tiem!
fig = plt.figure( figsize = (12,5))
plt.scatter(ds.time, ds.depth, c=ds.salinity, )
plt.colorbar(label='Salinity')
plt.ylim([1000, 0])
fig.autofmt_xdate()


# make a map

In [ ]:

# get a subset of the etopo data

lat_lims = [16, 18.9]
lon_lims = [-66, -61]

# grab some bathymetry

url = 'http://geoport.whoi.edu/thredds/dodsC/bathy/etopo1_bed_g2'

etopo1 = xr.open_dataset(url).sel(lat = slice(lat_lims[0], lat_lims[1]), lon = slice(lon_lims[0], lon_lims[1] ))
etopo1

In [ ]:

plt.figure( figsize=(10,10))
ax = plt.axes( projection = ccrs.Mercator() )
ax.coastlines(resolution='10m')

dpth_contours = np.append(np.arange(-5000, -500, 500), np.arange(-500, 0, 50)) 

plt.contourf( etopo1.lon, etopo1.lat, etopo1.topo, dpth_contours , transform=ccrs.PlateCarree(), cmap='Blues_r')

plt.plot(ds.longitude, ds.latitude, 'k.', transform = ccrs.PlateCarree() )
plt.plot(ds.longitude[-50:], ds.latitude[-50:], 'r.', transform = ccrs.PlateCarree() )


ax.set_extent((-65.5,-63.5, 17, 18.5) )

ax.gridlines(draw_labels=True)

In [ ]:
fig = plt.figure(figsize=(10.0, 5.0))

axes1 = fig.add_subplot(2, 2, 1)
axes2 = fig.add_subplot(2, 2, 2)

# axes1.plot(ds2.sci_water_temp, ds2.depth, '.')
axes1.scatter(ds2.sci_water_temp, ds2.depth, c=ds2.time)
axes1.set_ylim([100,0])
axes1.set_xlim([24,31])
axes1.set_ylabel('Depth')
axes1.set_xlabel('Temperature')

# axes2.plot(ds2.sci_water_temp, ds2.depth)
axes2.scatter(ds2.sci_water_temp, ds2.depth, c=ds2.time)
axes2.set_ylim([1000,900])
axes2.set_xlim([4,11])
axes2.set_ylabel('Depth')
axes2.set_xlabel('Temperature')